In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Concatenate, MaxPooling1D, Flatten, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/text_classification_with_CNN/

[Errno 2] No such file or directory: '/content/drive/MyDrive/text_classification_with_CNN/'
/content


In [ ]:
df = pd.read_csv('preprocessed_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessed_data.csv'

In [ ]:
df.head()

In [ ]:
#Source: andrew ng course: deeplearning specialization
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

#Embedding

In [ ]:
# prompt: print all the text having nan

df[df.isna().any(axis=1)]


In [ ]:
# prompt: drop all the rows having any column nan values

df.dropna(axis=0, how='any', inplace=True)


In [ ]:
# prompt: check nan value

df[df.isna().any(axis=1)]


In [ ]:
texts = df["text_combined"]

In [ ]:
# prompt: wants to know the distribution of the length of words in each texts

len_words = []
for text in texts:
    words = text.split()
    len_words.append(len(words))

# Plotting the histogram
# Define the bin width
bin_width = 500

# Plotting the histogram with specified bin width
bins = np.arange(min(len_words), max(len_words) + bin_width, bin_width)
plt.hist(len_words, bins=bins, edgecolor='black')
plt.title('Distribution of Numbers')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
label = df["Label"]

In [ ]:
# prompt: convert word_to_index to dictionary
word_to_index_dict = dict(word_to_index)

In [ ]:
len(word_to_index_dict)

In [ ]:
train_data = list()

for sent in tqdm(texts):
  d = list()
  for word in sent:

    if word in word_to_index_dict:
      d.append(word_to_index_dict[word])
  train_data.append(d)

In [ ]:
# prompt: pad train_data with 0 and make the equal length

from keras.preprocessing.sequence import pad_sequences

max_length = 500
train_data = pad_sequences(train_data, maxlen=max_length, padding='post')


In [ ]:
import sklearn.model_selection

# Split the data into train and test sets.
texts_train, texts_test, labels_train, labels_test = sklearn.model_selection.train_test_split(train_data, label, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets.
print(texts_train.shape)
print(texts_test.shape)
print(labels_train.shape)
print(labels_test.shape)

(15963, 500)
(3991, 500)
(15963,)
(3991,)


In [ ]:
#free variable
del train_data, texts, df, label

**Experiments:**

In [ ]:
NUM_FILTERS=16
EMBEDDING_DIM = 50
M=3
N=3
O=3
P=5
Q=5
R=5
S=3

In [ ]:
len(word_to_index_dict)

400000

In [ ]:
#Source chatgpt with detailed prompt, giving the complete information of the architecture one by one.

# Define the input shape
input_layer = tf.keras.layers.Input(shape=(500,), name='InputLayer')
# reshaped_input = tf.expand_dims(input_layer, axis=-1)

word_embedding_layer = tf.keras.layers.Embedding(input_dim=400000, output_dim=50, name='CharEmbedding')(input_layer)
transposed_input = tf.transpose(word_embedding_layer, perm=[0, 2, 1])  # Assuming you want to transpose the input

# Convolutional layers with different filter sizes
conv1 = Conv1D(filters=NUM_FILTERS, kernel_size=M, activation='relu')(transposed_input)
conv2 = Conv1D(filters=NUM_FILTERS, kernel_size=N, activation='relu')(transposed_input)
conv3 = Conv1D(filters=NUM_FILTERS, kernel_size=O, activation='relu')(transposed_input)

# Concatenate the convolutional layers
concatenated = Concatenate(axis=1)([conv1, conv2, conv3])

# Max pooling layer
maxpool1 = MaxPooling1D(pool_size=2)(concatenated)

# Convolutional layers with different filter sizes
conv4 = Conv1D(filters=NUM_FILTERS, kernel_size=P, activation='relu')(maxpool1)
conv5 = Conv1D(filters=NUM_FILTERS, kernel_size=Q, activation='relu')(maxpool1)
conv6 = Conv1D(filters=NUM_FILTERS, kernel_size=R, activation='relu')(maxpool1)

# Concatenate the convolutional layers
concatenated2 = Concatenate(axis=1)([conv4, conv5, conv6])

# Max pooling layer
maxpool2 = MaxPooling1D(pool_size=2)(concatenated2)

# Convolutional layer
conv7 = Conv1D(filters=NUM_FILTERS, kernel_size=S, activation='relu')(maxpool2)

# Flatten layer
flatten = Flatten()(conv7)

# Dropout layer
dropout = Dropout(rate=0.5)(flatten)

# Dense layer
dense = Dense(units=128, activation='relu')(dropout)

# Output layer
output = Dense(units=20, activation='softmax')(dense)

# Define the model
model = Model(inputs=input_layer, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
max_features =400000
embedding_dim = 50
sequence_length = 500

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, embedding_dim, input_length=sequence_length,\
                                    embeddings_regularizer = regularizers.l2(0.0005)))

model.add(tf.keras.layers.Conv1D(128,3, activation='relu',\
                                 kernel_regularizer = regularizers.l2(0.0005),\
                                 bias_regularizer = regularizers.l2(0.0005)))


model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(3, activation='sigmoid',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))




model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='Nadam', metrics=["CategoricalAccuracy"])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
encoded_labels = label_encoder.fit_transform(labels_train)

In [ ]:
texts_train.shape

(15963, 500)

In [ ]:
encoded_labels

array([15,  8,  8, ...,  9,  3, 19])

In [ ]:
history = model.fit(texts_train, encoded_labels,
                    epochs=30,
                    batch_size=32,
                    validation_split=0.1)

Epoch 1/30
449/449 [==============================] - 258s 575ms/step - loss: 1.7187 - accuracy: 0.4545 - val_loss: 2.7987 - val_accuracy: 0.2079
Epoch 2/30
449/449 [==============================] - 260s 580ms/step - loss: 1.5233 - accuracy: 0.5113 - val_loss: 2.8725 - val_accuracy: 0.2154
Epoch 3/30
449/449 [==============================] - 240s 536ms/step - loss: 1.3623 - accuracy: 0.5615 - val_loss: 3.0187 - val_accuracy: 0.2079
Epoch 4/30
449/449 [==============================] - 241s 536ms/step - loss: 1.2325 - accuracy: 0.5990 - val_loss: 3.3476 - val_accuracy: 0.1966
Epoch 5/30
449/449 [==============================] - 257s 572ms/step - loss: 1.1337 - accuracy: 0.6325 - val_loss: 3.5455 - val_accuracy: 0.2085
Epoch 6/30
449/449 [==============================] - 242s 538ms/step - loss: 1.0439 - accuracy: 0.6568 - val_loss: 3.5162 - val_accuracy: 0.2079
Epoch 7/30
449/449 [==============================] - 260s 579ms/step - loss: 0.9644 - accuracy: 0.6839 - val_loss: 3.7818 -

In [ ]:
# prompt: save model at current dir
model.save('cnn_model.keras')

In [ ]:
cd /content/drive/MyDrive/text_classification_with_CNN/

/content/drive/MyDrive/text_classification_with_CNN


In [ ]:
# prompt: load saved model
model = tf.keras.models.load_model('cnn_model.keras')

In [ ]:
# prompt: evaluate on the train data
train_results = model.evaluate(embedding_matrix_train, encoded_labels, batch_size=32)
print("Train accuracy:", train_results[1])

499/499 [==============================] - 42s 82ms/step - loss: 1.3042 - accuracy: 0.7526
Train accuracy: 0.7526154518127441


In [ ]:
# prompt: evaluate above model on test data?
encoded_labels_test = label_encoder.transform(labels_test)

# Evaluate the model on the test data
model.evaluate(embedding_matrix_test, encoded_labels_test)


125/125 [==============================] - 10s 81ms/step - loss: 1.5918 - accuracy: 0.6545


[1.5918047428131104, 0.6544725894927979]

**Observation:**

65% test accuracy and 75% train accuracy. It can be improved by doing better featurization but skipping since our goal was just experiment. And main goal is to implement the char lavel cnn.

**The end**